In this notebook I will attempt to analyse the performance of various machine learning algoriths and their predictive power on the S&P 500.

In [2]:
from datetime import datetime
import Methods as models
import Predictors as predictors
import stock_tools as st
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import gridspec

/home/ward_s/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ward_s/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Create a template with the available variables

In [3]:
interest = 'SPY'
start_date = datetime.strptime('2000-01-01', '%Y-%m-%d')
end_date = datetime.strptime('2010-12-31', '%Y-%m-%d')

In [4]:
# Get the data and correct for fluctuations
data = st.get_data(start_date,end_date,from_file=True)
corr_data = st.ohlc_adj(data)
# Create a predictors class which we will base our decisions from
pred = predictors.Predictors(corr_data)

The data is far too noisy to make accurate predictions. We apply a 5 day exponential rolling filter. This should preserve shape and reduce noise.

In [5]:
pred.e_filter(5)

In [6]:
def make_logic(x=0.):
    """
    Makes this signal if the Closing price has gone above a percentage
    
    :param x: Number describing the percentage change.
    :return: Pandas DataFrame containing the true/false of the percentage change.
    """
    return ((1 - pred.data.Close.shift(1).div(pred.data.Close)) > x)*1

As a start, we test all algorithms for applicability. Make an indicator for raise or fall. From one day to the next. In these cases the model is created on 0.8 of the previous years data and then tested on 20% of the data. A walk forward optimization is performed, using new train and test data. In the case of XGboost, walk forward optimization has not been performed as it is too computationally expensive.

The technical indicators which have been selected are:

'meanfractal'

'RStok0'

'mom'

'MACD_l'

These were chosen from running all indicators through xgboost and selecting those which contribute the most. See the file xgboost_test for the methodology.